# Table of Contents

1. [Introduction](#Introduction)
2. [Dependencies](#Dependencies)
3. [Reading Data](#Reading-Data)

# Introduction



# Dependencies 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objects as go
import plotly.express as px
import os

# Reading Data

In [2]:
print("Path of our data files:")
for path in os.listdir("./data"):
    print(os.path.join("./data", path))

Path of our data files:
./data\holidays_events.csv
./data\oil.csv
./data\sample_submission.csv
./data\stores.csv
./data\test.csv
./data\train.csv
./data\transactions.csv


### Train Data

- The training data, comprising time series of features `store_nbr`, `family`, and `onpromotion` as well as the target `sales`.
- `store_nbr` identifies the store at which the products are sold.
- `family` identifies the type of product sold.
- `sales` gives the total sales for a product `family` at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- `onpromotion` gives the total number of items in a product family that were being promoted at a store at a given date.

In [3]:
train_data = pd.read_csv("./data/train.csv", index_col="id")
train_data.head()

C:\Users\ubfil\anaconda3\envs\ai-env\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0
3,2013-01-01,1,BEVERAGES,0.0,0
4,2013-01-01,1,BOOKS,0.0,0


### Test Data

- The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
- The dates in the test data are for the 15 days after the last date in the training data.

In [4]:
test_data = pd.read_csv("./data/test.csv", index_col="id")
test_data.head()

,date,store_nbr,family,onpromotion
id,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0
3000889,2017-08-16,1,BABY CARE,0
3000890,2017-08-16,1,BEAUTY,2
3000891,2017-08-16,1,BEVERAGES,20
3000892,2017-08-16,1,BOOKS,0


### Sample Submission

- A sample submission file in the correct format.

In [5]:
sample_submission = pd.read_csv("./data/sample_submission.csv")
sample_submission.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


### Stores Data

- Store metadata, including `city`, `state`, `type`, and `cluster`.
- `cluster` is a grouping of similar stores.

In [6]:
stores_data = pd.read_csv("./data/stores.csv")
stores_data.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Oil Data

- Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

In [7]:
oil_data = pd.read_csv("./data/oil.csv")
oil_data.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


### Holiday Events Data

- Holidays and Events, with metadata
- **NOTE**: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
- Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

In [8]:
holidays_data = pd.read_csv("./data/holidays_events.csv")
holidays_data.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


### Additional Notes

- Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
- A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

All these data descriptions are not mine, they were provided by the Kaggle team. I put them here so we can have a better understanding of the data. You can check it out in the [Kaggle competition page](https://www.kaggle.com/c/store-sales-time-series-forecasting/data).

# Understanding the Data
In this section, we will look at the data and see what we can learn from it. We will merge different tables, create new features, and explore the data with different visualizations.

Firstly, let's merge our tables.

### Merging our tables

We will merge our training/test data with our store metadata, using the `store_nbr` column as the key.

In [10]:
train_data_stores = pd.merge(train_data, stores_data, on="store_nbr")
test_data_stores = pd.merge(test_data, stores_data, on="store_nbr")
train_data_stores.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


Now, let's do the same with our oil data. But firstly, let's see if our data has too many NaN values.

In [17]:
print("Shape of oil_data:" ,oil_data.shape)

Shape of oil_data: (1218, 2)


In [18]:
print("Number of null values in oil_data:")
oil_data.isnull().sum()

Number of null values in oil_data:


date           0
dcoilwtico    43
dtype: int64

We have only 43 NaN values in the oil data. But if we merge our table with the oil data, we will have way more NaN values. Let's analyze the best way to impute the NaN values and then merge our tables.

In [33]:
fig = go.Figure(go.Scatter(x=oil_data.date, y=oil_data.dcoilwtico, mode="lines"))
fig.update_layout(title="Oil Price ($)", xaxis_title="Date", yaxis_title="Price",
                  template="plotly_white", )
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ]),
        font_color="black"
    )
)
fig.show()

We can see that besides the oil price is unstable, the oil price is decreasing, and we can see sequence

In [12]:
train_data_oil = pd.merge(train_data_stores, oil_data, on="date")
train_data_oil.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN
